In [ ]:
!pip install spacy

In [1]:
import os
from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline
import sqlite3
import spacy
from PyPDF2 import PdfReader
from dotenv import load_dotenv

# Load environment variables (e.g., API keys from a .env file)
load_dotenv()
my_api_key = os.getenv("API_KEY")

# Import Groq API Client
from groq import Groq


# NLP-Based Query Parser #
class QueryParser:
    """
    Parses user queries using SpaCy to extract actionable tasks based on linguistic dependencies.
    """

    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")  # Load SpaCy's language model

    def parse(self, query):
        """
        Parses the input query and extracts tasks to analyze based on subject and object dependencies.
        :param query: String containing the user's query.
        :return: List of tasks derived from the query.
        """
        doc = self.nlp(query)
        tasks = [f"Analyze {token.text.lower()}" for token in doc if token.dep_ in ("nsubj", "dobj")]
        return tasks


# Extended Data Retrieval #
class SQLDataSource:
    """
    Handles retrieving data from an SQLite database.
    """

    def __init__(self, db_path, query):
        self.db_path = db_path
        self.query = query

    def fetch_data(self):
        """
        Fetches data by executing the provided SQL query on the database.
        :return: Dictionary containing SQL data.
        """
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(self.query)
        data = cursor.fetchall()
        conn.close()
        return {"sql_data": data}


class DocumentParser:
    """
    Handles extracting text data from PDF documents.
    """

    def __init__(self, file_path):
        self.file_path = file_path

    def fetch_data(self):
        """
        Extracts text from the specified file (only supports PDFs).
        :return: Dictionary containing document text.
        """
        if self.file_path.endswith(".pdf"):
            text = self._parse_pdf()
        else:
            raise ValueError("Unsupported file format.")
        return {"document_text": text}

    def _parse_pdf(self):
        """
        Parses the content of a PDF file.
        :return: Extracted text from the PDF.
        """
        reader = PdfReader(self.file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text


class APIDataSource:
    """
    Mock API data retrieval for competitor and market trends data.
    """

    def __init__(self, api_url):
        self.api_url = api_url

    def fetch_data(self):
        """
        Mock method to simulate API responses for different URLs.
        :return: Dictionary containing the simulated API response.
        """
        if "competitors" in self.api_url:
            return {"competitors": "Competitors in Region A offer durable products at competitive prices."}
        elif "market-trends" in self.api_url:
            return {"market_trends": "Growing demand for sustainable and durable products in Region A."}
        else:
            raise ValueError("Invalid API URL")


# Parallel Execution #
class ParallelReasoningPipeline:
    """
    Manages the execution of multiple reasoning tasks in parallel.
    """

    def __init__(self):
        self.steps = []

    def add_step(self, description, data_source, processor):
        """
        Adds a reasoning step to the pipeline.
        :param description: Description of the task.
        :param data_source: Data source object for fetching input data.
        :param processor: Processor object for analyzing the data.
        """
        self.steps.append({"description": description, "data_source": data_source, "processor": processor})

    def run(self):
        """
        Executes all steps in the pipeline in parallel.
        :return: List of results from all executed steps.
        """
        results = []

        def process_step(step):
            print(f"\n{step['description']}")
            data = step["data_source"].fetch_data()
            insights = step["processor"].process(data)
            print(f"Insights: {insights}")
            return insights

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_step, step) for step in self.steps]
            for future in futures:
                results.append(future.result())

        return results


# Enhanced Reasoning Logic #
class SalesProcessor:
    """
    Processes sales data retrieved from the database.
    """

    def process(self, data):
        return f"Sales data: {data['sql_data']}"


class FeedbackProcessor:
    """
    Processes customer feedback data retrieved from the document parser.
    """

    def process(self, data):
        return f"Customer feedback: {data['document_text'][:200]}..."  # Truncate for readability


class CompetitorProcessor:
    """
    Processes competitor insights data retrieved from the API.
    """

    def process(self, data):
        return f"Competitor insights: {data['competitors']}"


class MarketTrendsProcessor:
    """
    Processes market trend insights data retrieved from the API.
    """

    def process(self, data):
        return f"Market trends: {data['market_trends']}"


class SummarizationProcessor:
    """
    Summarizes insights using the Groq API.
    """

    def __init__(self, groq_client):
        self.groq_client = groq_client

    def process(self, insights):
        """
        Summarizes the combined insights using the Groq API.
        :param insights: List of insights to summarize.
        :return: Summarized text.
        """
        combined_text = " ".join(insights)

        # Use Groq API to summarize
        response = self.groq_client.chat.completions.create(
            model="llama3-70b-8192",  # Replace with a valid model name
            messages=[
                {"role": "system", "content": "Summarize the following insights"},
                {"role": "user", "content": combined_text},
            ],
        )

        # Extract summary from response
        if hasattr(response, "choices"):
            summary = response.choices[0].message.content
        else:
            summary = response["choices"][0]["message"]["content"]

        return summary


# Example Implementation #
def main():
    """
    Entry point for the program. Orchestrates query processing, reasoning steps, and summarization.
    """
    print(" Enhanced Multi-Hop Reasoning Agent \n")

    # Initialize Groq Client
    groq_client = Groq(api_key=my_api_key)  # Ensure API key is set in the environment

    # Parse the query
    query = "What are the key factors driving the decline in sales for Product X in the last quarter?"
    query_parser = QueryParser()
    tasks = query_parser.parse(query)
    print(f"Parsed Tasks: {tasks}")

    # Initialize the pipeline
    pipeline = ParallelReasoningPipeline()

    # Add reasoning steps
    pipeline.add_step(
        "Analyze sales data",
        SQLDataSource("sales.db", "SELECT * FROM sales WHERE product='Product X'"),
        SalesProcessor(),
    )
    pipeline.add_step(
        "Analyze customer feedback",
        DocumentParser("feedback.pdf"),
        FeedbackProcessor(),
    )
    pipeline.add_step(
        "Analyze competitor data",
        APIDataSource("mock://competitors"),
        CompetitorProcessor(),
    )
    pipeline.add_step(
        "Analyze market trends",
        APIDataSource("mock://market-trends"),
        MarketTrendsProcessor(),
    )

    # Run the pipeline
    insights = pipeline.run()

    # Summarize results
    summarizer = SummarizationProcessor(groq_client)
    final_summary = summarizer.process(insights)

    print("\nFinal Summary")
    print(final_summary)


# Run the Program #
if __name__ == "__main__":
    main()

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



 Enhanced Multi-Hop Reasoning Agent 

Parsed Tasks: ['Analyze factors', 'Analyze decline']

Analyze sales data
Insights: Sales data: [(1, 'Product X', 'Region A', 100), (2, 'Product X', 'Region B', 200)]

Analyze customer feedback

Analyze competitor data
Insights: Competitor insights: Competitors in Region A offer durable products at competitive prices.

Analyze market trends
Insights: Market trends: Growing demand for sustainable and durable products in Region A.
Insights: Customer feedback: Customer Feedback
Region A: Customers reported durability issues.
Region B: Customers are satisfied with the product....

Final Summary
Here are the summarized insights:

**Product Performance:**

* Product X sells 100 units in Region A and 200 units in Region B.

**Customer Feedback:**

* Region A: Customers unhappy with product durability.
* Region B: Customers satisfied with the product.

**Competitor Insights:**

* Competitors in Region A offer durable products at competitive prices.

**Mark

In [1]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import sqlite3
from PyPDF2 import PdfReader
from dotenv import load_dotenv
from rich.console import Console
from rich.panel import Panel

# Load environment variables
load_dotenv()
my_api_key = os.getenv("API_KEY")

# Import Groq API Client
from groq import Groq

# Initialize console for Rich output
console = Console()

class QueryParser:
    """Parses user queries using LLM to extract actionable tasks"""
    
    def __init__(self, groq_client):
        self.groq_client = groq_client

    def parse(self, query):
        """Extracts key analysis targets from user query"""
        prompt = f"""Analyze the following user query and identify the key factors or entities that need to be analyzed. 
        Return your response as a comma-separated list of terms, each term being a noun or noun phrase. 
        Do not include any additional text.

        Query: {query}
        Response:"""
        
        response = self.groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts key analysis targets from user queries."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
        )
        
        response_text = response.choices[0].message.content
        tasks = [f"Analyze {term.strip()}" for term in response_text.split(",") if term.strip()]
        return tasks


# Data Sources #
class SQLDataSource:
    """Handles retrieving data from an SQLite database."""
    def __init__(self, db_path, query):
        self.db_path = db_path
        self.query = query

    def fetch_data(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(self.query)
        data = cursor.fetchall()
        conn.close()
        return {"sql_data": data}


class DocumentParser:
    """Handles extracting text data from PDF documents."""
    def __init__(self, file_path):
        self.file_path = file_path

    def fetch_data(self):
        if self.file_path.endswith(".pdf"):
            text = self._parse_pdf()
        else:
            raise ValueError("Unsupported file format.")
        return {"document_text": text}

    def _parse_pdf(self):
        reader = PdfReader(self.file_path)
        text = "".join([page.extract_text() for page in reader.pages])
        return text


class APIDataSource:
    """Handles API data retrieval for competitor and market data."""
    def __init__(self, api_url):
        self.api_url = api_url

    def fetch_data(self):
        if "competitors" in self.api_url:
            return {"competitors": "Competitors in Region A offer durable products at competitive prices."}
        elif "market-trends" in self.api_url:
            return {"market_trends": "Growing demand for sustainable and durable products in Region A."}
        else:
            raise ValueError("Invalid API URL")


# Parallel Processing #
class ParallelReasoningPipeline:
    """Manages parallel execution of reasoning tasks with dynamic step numbering."""
    def __init__(self):
        self.steps = []

    def add_step(self, description, data_source, processor, data_type, color):
        """Adds an analysis step to the pipeline without predefined numbering."""
        self.steps.append({
            "description": description,
            "data_source": data_source,
            "processor": processor,
            "data_type": data_type,
            "color": color
        })

    def run(self):
        """Executes all pipeline steps in parallel and numbers them dynamically."""
        results = []
        step_number = 1  # Start numbering dynamically

        with ThreadPoolExecutor() as executor:
            future_to_step = {executor.submit(self.process_step, step): step for step in self.steps}

            for future in as_completed(future_to_step):
                step = future_to_step[future]
                insights = future.result()
                
                # Dynamically number the step
                step_content = (
                    f"[bold]Data Source:[/bold] {step['data_type']}\n"
                    f"[bold]Insights:[/bold] {insights}"
                )
                console.print(Panel(step_content, title=f"[bold]Step {step_number}: {step['description']}[/bold]", border_style=step["color"]))
                
                results.append(insights)
                step_number += 1  # Increment step number for next completed step

        return results

    def process_step(self, step):
        """Fetches data, processes it, and returns insights."""
        data = step["data_source"].fetch_data()
        return step["processor"].process(data)


# Data Processors #
class SalesProcessor:
    """Processes sales data using LLM for real insights."""
    def __init__(self, groq_client):
        self.groq_client = groq_client

    def process(self, data):
        if not data["sql_data"]:
            return "No sales data available."

        # Format data for LLM analysis
        sales_records = "\n".join([str(record) for record in data["sql_data"]])

        # Prompt LLM to analyze sales trends
        response = self.groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You are an expert business analyst."},
                {"role": "user", "content": f"Analyze the following sales data and provide insights:\n\n{sales_records}"}
            ],
            temperature=0,
        )
        return response.choices[0].message.content


class FeedbackProcessor:
    """Processes customer feedback using LLM for real insights."""
    def __init__(self, groq_client):
        self.groq_client = groq_client

    def process(self, data):
        feedback_text = data["document_text"]

        if not feedback_text.strip():
            return "No feedback data available."

        # Prompt LLM to analyze customer sentiment
        response = self.groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You are an expert in customer sentiment analysis."},
                {"role": "user", "content": f"Analyze the following customer feedback and provide insights:\n\n{feedback_text}"}
            ],
            temperature=0,
        )
        return response.choices[0].message.content


class CompetitorProcessor:
    """Processes competitor data using LLM for real insights."""
    def __init__(self, groq_client):
        self.groq_client = groq_client

    def process(self, data):
        competitor_info = data.get("competitors", "")

        if not competitor_info.strip():
            return "No competitor data available."

        # Prompt LLM to analyze competitor strategy
        response = self.groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You are an expert in competitor analysis."},
                {"role": "user", "content": f"Analyze the following competitor data and provide insights:\n\n{competitor_info}"}
            ],
            temperature=0,
        )
        return response.choices[0].message.content


class MarketTrendsProcessor:
    """Processes market trends using LLM for real insights."""
    def __init__(self, groq_client):
        self.groq_client = groq_client

    def process(self, data):
        market_trends_info = data.get("market_trends", "")

        if not market_trends_info.strip():
            return "No market trend data available."

        # Prompt LLM to analyze market trends
        response = self.groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You are an expert market analyst."},
                {"role": "user", "content": f"Analyze the following market trend data and provide insights:\n\n{market_trends_info}"}
            ],
            temperature=0,
        )
        return response.choices[0].message.content


class SummarizationProcessor:
    """Summarizes insights using LLM."""
    def __init__(self, groq_client):
        self.groq_client = groq_client

    def process(self, insights):
        """Generates final summary using LLM."""
        combined_text = " ".join(insights)
        
        response = self.groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "Create a concise, actionable summary of the following business insights."},
                {"role": "user", "content": combined_text}
            ]
        )

        return response.choices[0].message.content


def main():
    console.print("[bold cyan]\nEnhanced Multi-Hop Reasoning Agent[/bold cyan]\n")

    # Initialize Groq Client
    groq_client = Groq(api_key=my_api_key)

    # Parse the query using LLM
    query = "What are the key factors driving the decline in sales for Product X in the last quarter?"
    query_parser = QueryParser(groq_client)
    tasks = query_parser.parse(query)
    console.print(Panel(f"Parsed Tasks: {tasks}", title="Query Parsing", border_style="cyan"))

    # Initialize the pipeline
    pipeline = ParallelReasoningPipeline()

    # ✅ Remove hardcoded step numbers & let dynamic numbering handle it
    pipeline.add_step(
        "Retrieve and analyze sales data for Product X",
        SQLDataSource("sales.db", "SELECT * FROM sales WHERE product='Product X'"),
        SalesProcessor(groq_client),
        "SQL Database", "blue"
    )
    pipeline.add_step(
        "Retrieve and analyze customer feedback for Product X",
        DocumentParser("feedback.pdf"),
        FeedbackProcessor(groq_client),
        "Document", "green"
    )
    pipeline.add_step(
        "Retrieve and analyze competitor data",
        APIDataSource("mock://competitors"),
        CompetitorProcessor(groq_client),
        "API", "yellow"
    )
    pipeline.add_step(
        "Retrieve and analyze market trends",
        APIDataSource("mock://market-trends"),
        MarketTrendsProcessor(groq_client),
        "API", "red"
    )

    # Run the pipeline
    insights = pipeline.run()

    # Summarize results using LLM
    summarizer = SummarizationProcessor(groq_client)
    final_summary = summarizer.process(insights)

    console.print(Panel(final_summary, title="[bold]Final Summary[/bold]", border_style="magenta"))


if __name__ == "__main__":
    main()



Enhanced Multi-Hop Reasoning Agent

╭───────────────────────────────────────────────── Query Parsing ─────────────────────────────────────────────────╮
│ Parsed Tasks: ['Analyze Product X', 'Analyze sales', 'Analyze decline', 'Analyze quarter']                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────── Step 1: Retrieve and analyze sales data for Product X ─────────────────────────────╮
│ Data Source: SQL Database                                                                                       │
│ Insights: A great opportunity to dive into some sales data!                                                     │
│                                                                                                                 │
│ Based on the provided data, here are my initial insights:                                                       │
│                                                                                                                 │
│ **Product Performance:**                                                                                        │
│                                                                                                                 │
│ * We have only one product, 'Product X', in our dataset.                                                        │
│ * The sales data suggests that 'Product X' is performing well, with a total sales volume of 300 units (100 in   │
│ Region A + 200 in Region B).                                                                                    │
│                                                                                                                 │
│ **Regional Performance:**                                                                                       │
│                                                                                                                 │
│ * We have two regions, 'Region A' and 'Region B', in our dataset.                                               │
│ * 'Region B' is outperforming 'Region A' in terms of sales volume, with 200 units sold compared to 100 units in │
│ 'Region A'. This could indicate that 'Region B' has a stronger market demand for 'Product X' or that the sales  │
│ strategy in 'Region B' is more effective.                                                                       │
│                                                                                                                 │
│ **Key Questions and Areas for Further Analysis:**                                                               │
│                                                                                                                 │
│ * What are the prices of 'Product X' in each region? Are there any pricing differences that could be            │
│ contributing to the sales volume disparity?                                                                     │
│ * What are the marketing strategies and budgets for each region? Are there any differences in promotional       │
│ activities, advertising, or distribution channels that could be influencing sales?                              │
│ * Are there any seasonal or temporal factors that could be affecting sales in one or both regions?              │
│ * What are the customer demographics and preferences in each region? Are there any differences in customer      │
│ behavior or needs that could be driving sales?                                                                  │
│                                                                                                                 │
│ **Recommendations:**                                                                                            │
│                                                                                                                 │
│ * Consider allocating more resources to 'Region B' to capitalize on the existing demand and potentially         │
│ increase sales further.                                                                                         │
│ * Analyze the sales strategies and tactics in 'Region B' to identify best practices that could be applied to    │
│ 'Region A' to improve sales performance.              

╭───────────────────────── Step 2: Retrieve and analyze customer feedback for Product X ──────────────────────────╮
│ Data Source: Document                                                                                           │
│ Insights: Based on the customer feedback, here are some insights:                                               │
│                                                                                                                 │
│ **Overall Sentiment:** Mixed                                                                                    │
│                                                                                                                 │
│ **Regional Insights:**                                                                                          │
│                                                                                                                 │
│ * **Region A:** Negative sentiment (Durability issues)                                                          │
│         + Customers in Region A are experiencing problems with the product's durability, which suggests that    │
│ there may be a quality control issue or a design flaw that needs to be addressed.                               │
│         + This feedback indicates that the product may not be meeting customer expectations in terms of         │
│ longevity and reliability.                                                                                      │
│ * **Region B:** Positive sentiment (Satisfied with the product)                                                 │
│         + Customers in Region B are happy with the product, which suggests that it is meeting their needs and   │
│ expectations.                                                                                                   │
│         + This feedback indicates that the product is performing well in this region, and customers are likely  │
│ to continue using it and recommend it to others.                                                                │
│                                                                                                                 │
│ **Key Takeaways:**                                                                                              │
│                                                                                                                 │
│ 1. **Regional disparities:** There is a significant difference in customer sentiment between Region A and       │
│ Region B, which suggests that there may be regional-specific factors at play. This could be due to differences  │
│ in usage patterns, environmental conditions, or cultural preferences.                                           │
│ 2. **Product quality:** The durability issues reported in Region A suggest that there may be a problem with the │
│ product's quality or design. This needs to be investigated and addressed to prevent further customer            │
│ dissatisfaction.                                                                                                │
│ 3. **Customer expectations:** The positive feedback in Region B indicates that customers have certain           │
│ expectations from the product, which are being met. However, the negative feedback in Region A suggests that    │
│ these expectations are not being met in that region.                                                            │
│                                                                                                                 │
│ **Recommendations:**                                                                                            │
│                                                                                                                 │
│ 1. **Conduct a root cause analysis:** Investigate the durability issues reported in Region A to identify the    │
│ root cause of the problem.                            

╭───────────────────────────────── Step 3: Retrieve and analyze competitor data ──────────────────────────────────╮
│ Data Source: API                                                                                                │
│ Insights: Let's dive into the analysis of this competitor data.                                                 │
│                                                                                                                 │
│ **Key Takeaways:**                                                                                              │
│                                                                                                                 │
│ 1. **Product Durability**: Competitors in Region A are offering durable products, which suggests that they are  │
│ focusing on building a reputation for quality and reliability. This could be a key differentiator for them in   │
│ the market.                                                                                                     │
│ 2. **Competitive Pricing**: The fact that competitors are offering products at competitive prices implies that  │
│ they are trying to attract price-sensitive customers. This could indicate that the market is highly             │
│ competitive, and customers are looking for value for money.                                                     │
│                                                                                                                 │
│ **Insights:**                                                                                                   │
│                                                                                                                 │
│ 1. **Quality vs. Price Trade-off**: Competitors in Region A are trying to strike a balance between offering     │
│ durable products and keeping prices competitive. This suggests that customers in this region are willing to pay │
│ a premium for high-quality products, but only up to a certain point.                                            │
│ 2. **Market Saturation**: The presence of multiple competitors offering similar products at competitive prices  │
│ could indicate that the market is saturated. This might make it challenging for new entrants to gain traction   │
│ or for existing players to differentiate themselves.                                                            │
│ 3. **Customer Expectations**: Customers in Region A likely have high expectations for product quality and       │
│ durability. Meeting or exceeding these expectations could be crucial for competitors to gain customer loyalty   │
│ and retention.                                                                                                  │
│ 4. **Potential Opportunities**: Identifying areas where competitors are compromising on quality or price could  │
│ provide opportunities for differentiation. For example, offering even more durable products at a slightly       │
│ higher price point or providing exceptional customer service to justify a premium price.                        │
│                                                                                                                 │
│ **Recommendations:**                                                                                            │
│                                                                                                                 │
│ 1. **Conduct a thorough market analysis**: To better understand customer needs, preferences, and pain points in │
│ Region A.                                                                                                       │
│ 2. **Analyze competitors' product offerings**: Identify areas where competitors are compromising on quality or  │
│ price, and explore opportunities for differentiation.                                                           │
│ 3. **Develop a unique value proposition**: Focus on cr

╭────────────────────────────────── Step 4: Retrieve and analyze market trends ───────────────────────────────────╮
│ Data Source: API                                                                                                │
│ Insights: **Market Trend Analysis: Growing Demand for Sustainable and Durable Products in Region A**            │
│                                                                                                                 │
│ **Overview**                                                                                                    │
│                                                                                                                 │
│ The market trend data indicates a significant shift in consumer behavior in Region A, with a growing demand for │
│ sustainable and durable products. This trend is driven by increasing environmental awareness, changing          │
│ lifestyles, and a desire for products that not only perform well but also contribute to a more sustainable      │
│ future.                                                                                                         │
│                                                                                                                 │
│ **Key Insights**                                                                                                │
│                                                                                                                 │
│ 1. **Environmental Concerns**: The growing demand for sustainable products in Region A is largely driven by     │
│ consumers' increasing concerns about the environmental impact of their purchasing decisions. This is reflected  │
│ in the rise of eco-friendly and recyclable products, as well as a growing interest in products made from        │
│ sustainable materials.                                                                                          │
│ 2. **Durability Matters**: Consumers in Region A are seeking products that are built to last, reducing the need │
│ for frequent replacements and waste generation. This trend is driven by a desire for quality, reliability, and  │
│ value for money.                                                                                                │
│ 3. **Health and Wellness**: The demand for sustainable and durable products is also linked to the growing       │
│ importance of health and wellness in Region A. Consumers are seeking products that promote a healthy lifestyle, │
│ are free from harmful chemicals, and contribute to a cleaner environment.                                       │
│ 4. **Demographic Shift**: The trend is particularly pronounced among younger demographics (Millennials and Gen  │
│ Z) who are more likely to prioritize sustainability and durability in their purchasing decisions.               │
│ 5. **Competitive Advantage**: Companies that can effectively communicate their commitment to sustainability and │
│ durability are likely to gain a competitive advantage in Region A, as consumers are willing to pay a premium    │
│ for products that align with their values.                                                                      │
│                                                                                                                 │
│ **Opportunities and Recommendations**                                                                           │
│                                                                                                                 │
│ 1. **Product Innovation**: Companies should invest in research and development to create innovative,            │
│ sustainable, and durable products that meet the growing demand in Region A.                                     │
│ 2. **Sustainability Certification**: Obtaining certifications such as eco-labels or sustainability              │
│ certifications can help companies demonstrate their co

╭───────────────────────────────────────────────── Final Summary ─────────────────────────────────────────────────╮
│ Here is a concise, actionable summary of the business insights:                                                 │
│                                                                                                                 │
│ **Key Takeaways:**                                                                                              │
│                                                                                                                 │
│ 1. **Product Performance:** Product X is performing well, but there are regional differences in sales volume,   │
│ with Region B outperforming Region A.                                                                           │
│ 2. **Regional Insights:**                                                                                       │
│         * Region A: Negative customer sentiment due to durability issues, indicating a quality control problem  │
│ or design flaw.                                                                                                 │
│         * Region B: Positive customer sentiment, indicating the product meets customer expectations.            │
│ 3. **Competitor Analysis:**                                                                                     │
│         * Competitors in Region A focus on offering durable products at competitive prices, indicating a market │
│ where customers prioritize quality and value for money.                                                         │
│         * There may be opportunities for differentiation by offering even more durable products or exceptional  │
│ customer service.                                                                                               │
│ 4. **Market Trend Analysis:**                                                                                   │
│         * Growing demand for sustainable and durable products in Region A, driven by environmental awareness,   │
│ changing lifestyles, and a desire for products that contribute to a sustainable future.                         │
│                                                                                                                 │
│ **Recommendations:**                                                                                            │
│                                                                                                                 │
│ 1. **Conduct a root cause analysis:** Investigate durability issues in Region A and implement quality control   │
│ measures to address them.                                                                                       │
│ 2. **Analyze competitors' product offerings:** Identify areas where competitors compromise on quality or price, │
│ and explore opportunities for differentiation.                                                                  │
│ 3. **Develop a unique value proposition:** Focus on creating a compelling reason for customers to choose your   │
│ product over competitors', such as exceptional customer service or a more comprehensive warranty.               │
│ 4. **Innovate and certify sustainable products:** Invest in research and development to create innovative,      │
│ sustainable, and durable products that meet the growing demand in Region A, and obtain sustainability           │
│ certifications to demonstrate commitment to the environment.                                                    │
│ 5. **Optimize supply chains:** Minimize environmental impact and ensure responsible sourcing to reduce waste    │
│ and improve credibility with environmentally conscious customers.                                               │
│                                                                                                                 │
│ **Actionable Next Steps:**                            